Задание:
1. Снять видео на камеру (любую, можно телефона). Выполнить кручение видео с скоростью 10 градусов в секунду относительно центра (в любую сторону). 
2. Видео после выполнения п.1 и вернуть его в исходное состояние. 
3. Сконвертируйте видео в mp4 при помощи ffmpeg

Для выполнения задания необходимо:
1. Загрузить исходное видео. Будет использовано видео в формате hevc.
2. Применить поворот к исходному видео (скорость - 10 градусов в секунду). Целесообразно привязыватся к fps видео, для синхронизации обратного вращения в дальнейшем.
3. Выполнить поворот в обратную сторону для устранения вращения. 
4. Сохранить полученное видео.

In [1]:
import cv2
import numpy as np
import pandas as pd

Функия read_and_save_rotate открывает видео и выполняет его вращение с помощью функции rotate_function, переданную в качестве аргумента.

In [63]:
def read_and_save_rotate (read_name,out_name,out_format,angle,rotate_function, DEBUG = 1): 
  cap = cv2.VideoCapture(read_name)


  output_filename = out_name
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))



  fourcc = cv2.VideoWriter_fourcc(*out_format)
  out = cv2.VideoWriter(output_filename, fourcc, fps, (512, 640))


  rotate_angle = 0
  frame_counter = 0


  if (cap.isOpened()== False): 
    print("Ошибка открытия видеопотока")
    


  while(cap.isOpened()):

      
    ret, frame = cap.read()
      
     # Считаем кадры и каждую секунду меняем угол наклона изображения 
    if frame_counter%fps == 0:
        rotate_angle += angle
      
    if rotate_angle == 360:
        rotate_angle == 0

      
      
    if ret == True:
        
      frame_counter+=1
      
      frame = rotate_function(frame, rotate_angle) # Вызываем функцию вращения кадра
      out.write(frame)
      if DEBUG:

        cv2.imshow('window',frame)
      

      if cv2.waitKey(25) & 0xFF == ord('q'):
        break
        
    

    else: 
      break
  
  # Освобождаем ресурсы
  cap.release()
  out.release()

  cv2.destroyAllWindows()

Функция rotate_image_border принмает кадр и угол, на который нужно его повернуть. Также функция масштабирует изображение, для исключения его обрезки при выводе.

In [53]:
def rotate_image_border(image, angle):
    center = (image.shape[1] // 2, image.shape[0] // 2)  # Центр вращения
    height, width = image.shape[:2]
    diag = int(np.sqrt(width**2 + height**2))  # Диагональ (наибольшая сторона после вращения)

    # Добавление чёрных рамок, чтобы вместить всё изображение после поворота
    border_x = (diag - width) // 2
    border_y = (diag - height) // 2
    image_with_border = cv2.copyMakeBorder(image, border_y, border_y, border_x, border_x, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    # Обновление центра вращения из-за рамок
    new_center = (image_with_border.shape[1] // 2, image_with_border.shape[0] // 2)
    
    rot_mat = cv2.getRotationMatrix2D(new_center, angle, 1.0)
    

# Применение вращения
    rotated_image = cv2.warpAffine(image_with_border, rot_mat, (diag, diag), flags=cv2.INTER_LINEAR)
    frame = cv2.resize(rotated_image, (512, 640))
    return frame

Функция rotate_image_reverse получает кадр и угол, на который он повернут. Вычисляет обратную матрицу поворота и применяет ее к изображению.

In [81]:
def rotate_image_reverse(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  h, w = image.shape[:2]
  

  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  
  rot_mat_3x3 = np.vstack([rot_mat, [0, 0, 1]])
# Находим обратную матрицу
  inv_rot_mat_3x3 = np.linalg.inv(rot_mat_3x3)
# Возвращаем обратно в 2x3
  inv_rot_mat = inv_rot_mat_3x3[:2, :]
  
  result = cv2.warpAffine(image, inv_rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

In [64]:
read_and_save_rotate(read_name='IMG_6210.hevc',out_name='output.mov',out_format='Moov',angle = 10, DEBUG = 1, rotate_function=rotate_image_border)


In [85]:
read_and_save_rotate (read_name='output.mov',out_name='original_convert.mp4',out_format='mp4v',angle = 10, DEBUG = 1, rotate_function=rotate_image_reverse)

В ходе работы осуществлено вращение изображения против часовой стрелки со скоростью 10 градусов в секунду и компенсация этого вращения. 
Вращение картинки удалось скомпенсировать, однако, в результате вращения и масштабирования появляются артефакты. При компенсации вращения видно смещение краев кадра.

<video width="640" height="512" controls>
  <source src="original_convert.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
